In [1]:
import pandas as pd 
import glob
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
vectorizer = TfidfVectorizer()

In [3]:
L = []
for csv in glob.glob('*.csv'):
    df1 = pd.read_csv(csv, encoding='latin-1')
    L.append(df1)
    
df = pd.concat(L, axis = 0, sort=False)


In [4]:
df['Euroskepticism_Score'].unique()
#checking how many unique vals are there per column so can we can set all non zero vals to 1

array([0. , 0.5, 1. , nan])

In [5]:
df['Nationalism_Score'].unique()

array([0.5, 0. , 1. , nan])

In [6]:
df['Populism_Score'].unique()

array([0.0, 0.5, nan,
       ' Gaddafi attacked the French for ???interfering in his business?? ',
       '0', '0.5', '1',
       'EU must take appropriate and swift action to protect and guard its aquaculture against such unfair competition.',
       1.0], dtype=object)

In [7]:
df = df[['Question Text', 'Euroskepticism_Score', 'Nationalism_Score','Populism_Score']]

In [8]:
P= [' Gaddafi attacked the French for ???interfering in his business?? ',
    'EU must take appropriate and swift action to protect and guard its aquaculture against such unfair competition.']

df = df.dropna()
df = df[~df['Populism_Score'].isin(P)]

In [9]:
df

,Question Text,Euroskepticism_Score,Nationalism_Score,Populism_Score
0,In her recent statements Ms Maria Fekter the A...,0.0,0.5,0
1,How would the High Representative characterise...,0.0,0.0,0
2,How would the High Representative characterise...,0.0,0.0,0
3,How would the High Representative characterise...,0.0,0.0,0
4,In his speech to the European Economic and Soc...,0.5,0.0,0
5,In view of the important contribution made by ...,0.0,0.0,0
6,The financial support measures granted to some...,0.5,0.0,0.5
7,In 2009 the European Union launched the School...,0.0,0.0,0
8,During 2010 three wolves were killed on the Od...,0.0,0.0,0
9,Is the European Commission aware of the pheno...,0.0,0.0,0


In [10]:
X = vectorizer.fit_transform(df['Question Text'])

In [11]:
X

<1579x14063 sparse matrix of type '<class 'numpy.float64'>'
	with 157651 stored elements in Compressed Sparse Row format>

In [12]:
def make_one(x):
    if x > 0:
        return 1
    else:
        return 0

In [13]:
for w in ['Euroskepticism_Score','Nationalism_Score', 'Populism_Score']:
    df[w] = df[w].astype('float')
    df[w] = df[w].apply(make_one)

In [14]:
df['Euroskepticism_Score'].unique()

array([0, 1], dtype=int64)

In [15]:
df['Nationalism_Score'].unique()

array([1, 0], dtype=int64)

In [16]:
df['Populism_Score'].unique()

array([0, 1], dtype=int64)

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, df['Euroskepticism_Score'], test_size = .3)

In [37]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [38]:
y_pred = gbc.predict(X_test)

In [39]:
accuracy_score(y_test, y_pred)

0.8375527426160337

In [40]:
confusion_matrix(y_test, y_pred)

array([[384,   9],
       [ 68,  13]], dtype=int64)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, df['Nationalism_Score'], test_size = .3)

In [42]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [43]:
y_pred = gbc.predict(X_test)

In [44]:
accuracy_score(y_test, y_pred)

0.9156118143459916

In [45]:
confusion_matrix(y_test, y_pred)

array([[428,  12],
       [ 28,   6]], dtype=int64)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, df['Populism_Score'], test_size = .3)

In [47]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [48]:
y_pred = gbc.predict(X_test)

In [49]:
accuracy_score(y_test, y_pred)

0.9282700421940928

In [50]:
confusion_matrix(y_test, y_pred)

array([[435,  14],
       [ 20,   5]], dtype=int64)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, df['Euroskepticism_Score'], test_size = .3)

In [59]:
confusion_matrix(y_test, y_pred)

array([[394,  18],
       [ 61,   1]], dtype=int64)

In [56]:
import keras

Using TensorFlow backend.


In [57]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
batch_size = 32
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2


print('Build model...')
model = Sequential()


# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))


Build model...


ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=  (0, 8145)	0.15200255909896948
  (0, 5028)	0.15200255909896948
  (0, 5846)	0.15200255909896948
  (0, 6361)	0.15200255909896948
  (0, 8845)	0.15200255909896948
  (0, 14030)	0.15200255909896948
  (0, 5378)	0.30400511819793896
  (0, 5895)	0.15200255909896948
  (0, 8816)	0.43795441795976225
  (0, 13605)	0.12482406947332395
  (0, 2840)	0.12300395958524686
  (0, 8630)	0.13427872496149681
  (0, 2913)	0.13427872496149681
  (0, 3702)	0.11571805651998936
  (0, 13604)	0.12482406947332395
  (0, 7043)	0.08671945700626674
  (0, 7626)	0.11129787856015626
  (0, 7554)	0.0983487743934464
  (0, 10489)	0.0983487743934464
  (0, 711)	0.12300395958524686
  (0, 4068)	0.12132962079478696
  (0, 4070)	0.08968920056499602
  (0, 9024)	0.08968920056499602
  (0, 9097)	0.08734392331512207
  (0, 8220)	0.09400536007152425
  :	:
  (1104, 6518)	0.023347594386741364
  (1104, 2464)	0.043937780504260664
  (1104, 13835)	0.047